# Classifying U.S. Presidents Speeches
The goal of this notebook is to evaluate a ML model that predicts which U.S. president read the speech.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import seaborn as sns
import string
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score

## Data load and exploratory data analysis

In [ ]:
df = pd.read_csv('/kaggle/input/us-state-of-the-union-addresses-1790-2019/state_ofthe_union_texts.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
len(df['President'].unique())

In [ ]:
df['President'].unique()

### Calculating text length and word count
Maybe has some interesting information

In [ ]:
df['Text Length'] = df['Text'].apply(len)
df['Word Count'] = df['Text'].apply (lambda t: len(nltk.tokenize.wordpunct_tokenize(t)))

In [ ]:
df.head()

In [ ]:
sns.distplot(df['Text Length'], bins=10)
plt.title("Speeches's Text Length Frequency Distribution")

In [ ]:
pd.cut(df['Text Length'], bins=10).value_counts()

In [ ]:
sns.distplot(df['Word Count'], bins=10)
plt.title("Speeches's Word Count Frequency Distribution")

In [ ]:
pd.cut (df['Word Count'], bins=10).value_counts()

In [ ]:
sns.distplot(df['Year'], bins=10)
plt.title("Speeches's Year Frequency Distribution")

In [ ]:
pd.cut (df['Year'], bins=10).value_counts()

In [ ]:
plt.figure(figsize=(15,10))
ax = sns.boxplot(x='President', y='Text Length', data=df, palette='inferno')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.title('Text Length Plot by President')

In [ ]:
g = sns.factorplot('President', data=df, kind='count', aspect=3, palette='inferno')
g.set_xticklabels(rotation=90)
plt.title('Number of Speeches by President')

### Drop the labels that have less than 2 speeches

In [ ]:
df = df[(df['President'] != 'Zachary Taylor') & (df['President'] != 'Warren G. Harding') & (df['President'] != 'Ronald Reagan')]

## Model creation

In this step, it was tried with CountVectorizer and TfidfVectorizer. Besides that, these vectorizer was tested with and without stemmer. For machine learning model, it was tested with naive bayes, random forests, adaboost, and linear SVM.

The chosen model uses the CountVectorizer without stemming, and dropping stopwords and accents, and using linear SVM with penalty l1.

In [ ]:
pipeline = Pipeline([
    ('cv', CountVectorizer(stop_words='english', strip_accents='ascii')),
    ('lsvm', SGDClassifier(loss='hinge', verbose=0, penalty='l1'))
])

Here I will create a train dataset with 2 different randomly selected speeches for each label. This is necessary because the train_test_split function create train dataset that doesn't have some label samples, reducing the prediction performance.

In [ ]:
data_train= []
for p in df['President'].unique():
    text1 = df[df['President'] == p].sample(1)['Text'].values[0]
    data_train.append([p, text1])
    text2 = ''
    while True:
        text2 = df[df['President'] == p].sample(1)['Text'].values[0]
        if text2 != text1:
            break
    data_train.append([p, text2])
df_train= pd.DataFrame(data_train, columns=['President', 'Text'])

In [ ]:
%time pipeline.fit(df_train['Text'], df_train['President'])

In [ ]:
%time preds = pipeline.predict(df['Text'])

In [ ]:
confusion_matrix(df['President'], preds)

In [ ]:
print (classification_report(df['President'], preds))

In the begining, the classification report had some zero precision labels. With the random selection of speeches to generate the train set, there's no more zero precision labels.